In [50]:
import numpy as np
import math
import random 
# Define X gate :

X = np.array([
[0, 1],
[1, 0]
])

# Define Z gate :

Z = np.array([
[1, 0],
[0,-1]
])

# Define H gate:

H = np.array([
[1/np.sqrt(2), 1/np.sqrt(2)],
[1/np.sqrt(2), -1/np.sqrt(2)]
])
    

# Define 2x2 Identity

I = np.identity(2)


# Define projection operator |0><0|

P0x0 = np.array([
[1, 0],
[0, 0]
])

# Define projection operator |1><1|

P1x1 = np.array([
[0, 0],
[0, 1]
])


In [60]:
def get_ground_state(num_qubits):
    
    # all qubits are at |0> in ground state hence initializing the first qubit and subsequent qubits to be combined 
    
    q0 = [1, 0]
    combined_state = [1,0]
    
    #combining n qubits one after another to form a 2**n state with with only the state for which all qubits are 0 has the wieght '1'
    
    for i in range(num_qubits-1):
        combined_state = np.kron(combined_state, q0)
    return combined_state

def get_operator(total_qubits, gate_type, target_qubits):
    # initializing the main operator and the case operators for controled gate situation
        O = [1]
        I0 = [1]
        I1 = [1]
        
    # getting the gate length i.e no. of qubits on which it will be applied 
        qubitgate = [char.upper() for char in gate_type]
        c = len(qubitgate)
        
    #checking the gate or controller for each respective qubit in the target and getting the operator
        for j in range(total_qubits):
            if(j in target_qubits):
                #checking for if the operator to be calculated has only one gate involved
                if(c==1 and qubitgate[0]=='X'):
                    O = np.kron(O,X)
                elif(c==1 and qubitgate[0]=='H'):
                    O = np.kron(O,H)
                elif(c==1 and qubitgate[0]=='Z'):
                    O = np.kron(O,Z)
                #checking for if the operator to be calculated has multiple gates/controllers involved
                elif(qubitgate[target_qubits.index(j)]=='H' and c!=1):
                    I0 = np.kron(I0,I)
                    I1 = np.kron(I1,H)
                elif(qubitgate[target_qubits.index(j)]=='X' and c!=1):
                    I0 = np.kron(I0,I)
                    I1 = np.kron(I1,X)
                elif(qubitgate[target_qubits.index(j)]=='Z' and c!=1):
                    I0 = np.kron(I0,I)
                    I1 = np.kron(I1,Z)
                elif(qubitgate[target_qubits.index(j)]=='C' and c!=1):
                    I0 = np.kron(I0,P0x0)
                    I1 = np.kron(I1,P1x1)
                #if the input is erroneous
                else:
                    print("input error")
                    exit(0)
            # for the gates not in the target qubits but will show in operator
            else:
                if(c==1):
                    O = np.kron(O,I)
                else:
                    I0 = np.kron(I0,I)
                    I1 = np.kron(I1,I)
        #if there was a controller qubit
        if (c!=1):
            O = I0 + I1
            
    #returning the operator
        return O

def run_program(initial_state, program):
    # getting the gates and the targets for respective gates from the program
    gate = [ sub['gate'] for sub in program ]
    target = [ t['target'] for t in program]
    
    #applying the gate operators one by one and getting the new state
    new_state = initial_state
    for i in range(len(gate)):
        new_state = np.dot(new_state,get_operator(int(math.log2(len(initial_state))),gate[i],target[i]))
    
    #after applying all the gate operators the new state is final state
    return new_state

def measure_all(state_vector,num_shots):
    # marking different possible states to their weights obtaines in final state
    different_states = dict()
    for t in range(len(state_vector)):
        different_states[str(np.binary_repr(t, width=int(math.log2(len(state_vector)))))] = state_vector[t]
        
    #doing the measurement for num_shot times
    states = list(different_states.keys())
    values = [i**2 for i in different_states.values()]
    measurement = random.choices(states,weights =values,k=num_shots) 
    return measurement

def get_counts(state_vector, num_shots):
    
    #initializing every state's count to 0
        count = dict()
        for t in range(len(state_vector)):
            if(state_vector[t]!=0):
                count[str(np.binary_repr(t, width=int(math.log2(len(state_vector)))))] = 0
                
    #doing the measurements of num_shots
        measure = measure_all(state_vector,num_shots)
        
    #counting total occurences each state in the measurements 
        for i in measure:
             for j in list(count.keys()):
                    if(j==i):
                        count[j] = count[j] + 1
        return count

In [61]:
# Define program:

# should be of the format "program =[{ "gate": "__...", "target": [_,_...] }, {...}, ...]""
# value of gate can be anything ccccx , cxxx etc but should be string and involving x z or h
# value of target should be the target qubits (starting with 0) for the respective operation in the string in order
# there can be any no. of gates applied ,example is given below:
my_circuit=[{ "gate": "x", "target": [0] },{ "gate": "z", "target": [0] }]

num_qubit = 2
# Create "quantum computer" with num_qubits (this is actually just a vector :) )


my_qpu = get_ground_state(num_qubit)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(final_state, 1000)

print(counts)


{'10': 1000}
